In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [15]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [16]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [17]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [18]:
def R7_test():
    print("Test-R7: Sum of validator and delegator earning should match the block reward")
    num = int(input("enter the number of blocks you want to test: "))
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    print("current block", current_block, "next block", next_block)
    
    acc_rewards_prev = dict()
    delegations_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            ds = i['validator']['delegations']
            dels = dict()
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                dels[d_addr] = d_reward
            delegations_prev[address] = dels
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
    
    iterations = 0
    while iterations < num:
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        delegations_curr = dict()
        validator_infos = getAllValidatorInformation()
        block_reward = 28e18
        validator_rewards = 0
        total_reward = 0
        signers = 0
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                signers = signers + 1
                # block reward of the validator
                shard_metrics = i['metrics']['by-shard']
                validator_reward = 0
                for by_shard in shard_metrics:
                    validator_addr = by_shard['earning-account']
                    by_shard_reward = block_reward * float(by_shard['overall-percent']) / 0.32
                    validator_reward = validator_reward + by_shard_reward
                
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                reward = reward_accumulated
                if address not in acc_rewards_prev:
                    continue
                reward = reward_accumulated - acc_rewards_prev[address]
                # this reward should match sum of delegation rewards
                ds = i['validator']['delegations']
                del_rewards = 0
                dels = delegations_prev[address]
                for d in ds:
                    d_addr = d['delegator-address']
                    d_reward = d['reward']
                    del_rewards = del_rewards + d_reward
                    if d_addr in dels:
                        del_rewards = del_rewards - dels[d_addr] 
                if del_rewards != reward:
                    print("Test R7: Fail for validator", address, \
                          "expected block reward", validator_reward, \
                          "validator block reward", reward, \
                          "delegation reward", del_rewards)           
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        delegations_prev = delegations_curr
        
        iterations = iterations + 1
    print("done!")

In [19]:
R7_test()

Test-R7: Sum of validator and delegator earning should match the block reward
enter the number of blocks you want to test: 1
current block 770 next block 771
Test R7: Fail for validator one10jvjrtwpz2sux2ngktg3kq7m3sdz5p5au5l8c8 expected block reward 4.3221476510067113e+18 validator block reward 4322147651006711400 delegation reward 4322147651006711404
Test R7: Fail for validator one1y5n7p8a845v96xyx2gh75wn5eyhtw5002lah27 expected block reward 1.1042039206878306e+19 validator block reward 11042039206878306704 delegation reward 11042039206878306705
Test R7: Fail for validator one1c9h3u72czs6sk755tjyse7x5t70m38ppnkx922 expected block reward 4.61685815014006e+18 validator block reward 4616858150140059936 delegation reward 4616858150140059937
done!
